In [1]:
import pickle
import numpy as np
import scipy.sparse as sparse
from IPython.display import display
import implicit
from modules.evaluate import evaluate_system
from modules.model_content_based import ContentBasedModel
from modules.get_data import GetData, list_click_article_by_user, add_features_model_cont_based, add_features_model_collab
import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext pycodestyle_magic

In [3]:
# %flake8_on

<div class="alert alert-block alert-info">
<h2><center><strong> Systeme de recommandation</strong></center></h2>
        
</div>

Il existe 2 types de système de recommandation :

- un basé sur "Content-based" 

Le système va suggéré des elements similaires à ceux déjà vu par l'utilisateur

- un basé sur "Collaborative filtering"

Le système va essayer de prédire les préférences d'un utilisateur en se basant sur les préférences semblables d'utilisateur.
Il fonctionne en recherchant un grand groupe de personnes et en trouvant un plus petit nombre d'utilisateurs ayant des goûts similaires à ceux d'un utilisateur particulier. Il examine les éléments qu'ils aiment et les combinent pour créer une liste classée de suggestions.

<div class="alert alert-block alert-success">  
<strong>1.1 Chargement du jeu de données<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

*On charge le jeu de données provenant du News "Portal"*

In [4]:
DATA_PATH_INPUT = "data/data_source/"
DATA_PATH_OUTPUT = "data/data_output/"
data_df = GetData(
                  interaction_path=DATA_PATH_INPUT+"clicks/",
                  article_path=DATA_PATH_INPUT+"articles_metadata.csv",
                  matrice_embedding_path=DATA_PATH_INPUT+"articles_embeddings.pickle"
                   )
interactions_df, articles_df, article_matrice = data_df.load_data()

On transforme la valeur ancienneté pour avoir le nombre de jours par rapport à l'article le plus récent : 

In [5]:
articles_df['anciennete'] = abs(articles_df['created_at_ts'] - articles_df['created_at_ts'].max())
articles_df['anciennete'] = articles_df['anciennete'] / np.timedelta64(1, 'D')

Ajout de la popularité d'un article (nombre de clicks totaux) : 

In [6]:
article_pop_df = interactions_df[['click_article_id', 'user_id']]
article_pop_df = article_pop_df.groupby(by=['click_article_id'],  as_index=False).agg('count')
article_pop_df.rename(columns={"user_id": "popularite"}, inplace=True)
article_pop_df = article_pop_df.sort_values(by=['popularite'], ascending=False)

In [7]:
articles_df.sort_values(by=['created_at_ts'], ascending=False, inplace=True)
display(articles_df.head())

article_id  category_id created_at_ts  publisher_id  words_count  \
294012      294012          428    2018-12-02             0           70   
17017        17017            7    2018-12-02             0          144   
21484        21484           10    2018-12-02             0          232   
89868        89868          199    2018-12-02             0          182   
21487        21487           10    2018-12-02             0          162   

        anciennete  
294012         0.0  
17017          0.0  
21484          0.0  
89868          0.0  
21487          0.0

- Ajout de la popularite et de l'ancienneté au jeu de données initial : 

In [8]:
articles_new_features_df = articles_df.merge(
                                            article_pop_df,
                                            how='left',
                                            left_on=['article_id'],
                                            right_on=['click_article_id']
                                          )
articles_new_features_df.drop(['click_article_id'], axis=1, inplace=True)
articles_new_features_df['popularite'] = articles_new_features_df['popularite'].fillna(0)
articles_new_features_df.sort_values(by=['article_id'], ascending=False, inplace=True)
display(articles_new_features_df.head())

article_id  category_id created_at_ts  publisher_id  words_count  \
112272      364046          460    2017-09-19             0          479   
44458       364045          460    2018-01-14             0          126   
247898      364044          460    2016-03-14             0          177   
264568      364043          460    2015-12-06             0          463   
269000      364042          460    2015-11-06             0          144   

        anciennete  popularite  
112272       439.0         2.0  
44458        322.0         0.0  
247898       993.0         0.0  
264568      1092.0         8.0  
269000      1122.0         0.0

In [9]:
articles_new_features_df = articles_new_features_df.sort_values(by=['popularite'], ascending=False)

- permet de connaitre les articles consultés pour chaque utilisateur : 

In [10]:
interactions_user_df = list_click_article_by_user(interactions_df)

- ajout d'information à la matrice embedding (nombre de mots/article) :

In [11]:
article_matrice= add_features_model_cont_based(article_matrice, articles_df)

- permet de connaitre la force d'interraction entre un utilisateur et un article . Remplace le rating dans le cas d'une interraction implicite. Il est basé sur le nombre de clicks effectués par l'utilisateur au cours de la période concernée.

In [12]:
collab_data_df = add_features_model_collab(articles_df, interactions_df)
collab_data_df.head()

user_id  article_id  interactionStrength
0        0       68866                    1
1        0       87205                    1
2        0       87224                    1
3        0       96755                    1
4        0      157541                    1

 <div class="alert alert-block alert-success">  
<strong>1.2 Content Based Model<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

- On sauvegarde les informations pour le modèle sous forme pickle :

In [13]:
FILENAME = DATA_PATH_OUTPUT + 'article_matrice.pkl'
outfile = open(FILENAME, 'wb')
pickle.dump(article_matrice, outfile)
outfile.close()

FILENAME = DATA_PATH_OUTPUT + 'interactions_user.pkl'
outfile = open(FILENAME, 'wb')
pickle.dump(interactions_user_df, outfile)
outfile.close()


- Sélection de 5 articles les plus proche en utilisant la distance cosninus pour l'utilisateur 10

In [14]:
content_based_model = ContentBasedModel()
recommendations = content_based_model.get_recommandations(10)

In [15]:
display(recommendations)

[34328, 34175, 34224, 34121, 34339]

 <div class="alert alert-block alert-success">  
<strong>1.3 Colaborative Based Model<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

On calcule les matrices factorisations pour le modèle ALS (Alternating Least Squares)

In [16]:
sparse_item_user = sparse.csr_matrix((collab_data_df['interactionStrength'].astype(float),
                                     (collab_data_df['article_id'], collab_data_df['user_id'])))
sparse_user_item = sparse.csr_matrix((collab_data_df['interactionStrength'].astype(float),
                                     (collab_data_df['user_id'], collab_data_df['article_id'])))

Construction du modèle en se basant sur l'algorithme ALS

In [17]:
colab_model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)
ALPHA_VAL = 40
data_conf = (sparse_item_user * ALPHA_VAL).astype('double')
colab_model.fit(data_conf)

  0%|          | 0/20 [00:00<?, ?it/s]

- Sauvegarde des données nécessaires sous forme de pickle

In [26]:

FILENAME = DATA_PATH_OUTPUT + 'colab_model.pkl'
outfile = open(FILENAME, 'wb')
pickle.dump(colab_model, outfile)
outfile.close()

FILENAME = DATA_PATH_OUTPUT + 'sparse_item_user.pkl'
outfile = open(FILENAME, 'wb')
pickle.dump(sparse_item_user, outfile)
outfile.close()

- Sélection de 5 articles les plus proche en utilisant l'algorithme basé moindre carré pour l'utilisateur 10

In [21]:
# Create recommendations for person with id 50
USER_ID = 10
recommendations = colab_model.recommend(USER_ID,
                                        sparse_user_item,
                                        N=5,
                                        filter_already_liked_items=True)

In [22]:
recommendations_list = []
for i, j in recommendations:
    recommendations_list.append(i)
display(recommendations_list)

[129434, 124177, 336220, 289090, 124749]

 <div class="alert alert-block alert-success">  
<strong>1.4 Metriques<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

Pour calculer les performances de chaque modèle, on utilisera :
   - l'ancienneté des articles recommandés 
   - leur popularité
   - la catégorie
   - la taille du modèle

In [27]:
USER_SIZE = 1000
NB_USERS = interactions_user_df['user_id'].nunique()
users_list = np.random.randint(0,NB_USERS-1, size=USER_SIZE)

In [29]:
total_cat, pourcentage_cat, moyenne_anciennete, moyenne_popularite = \
            evaluate_system(model_name='RANDOM',
                            model = None,
                            sparse_user_item=None,
                            interactions_user_df=interactions_user_df,
                            articles_new_features_df=articles_new_features_df,
                            users_size=USER_SIZE,
                            users=users_list)

print(f'Il y a au total {total_cat} catégories identiques ce qui repréente (sur les recommandations) {pourcentage_cat}%')
print(f'En moyenne, l ancienneté est de {moyenne_anciennete} jours et la popularite de {moyenne_popularite} clicks')

Il y a au total 790 catégories identiques ce qui repréente (sur les recommandations) 8.7%
En moyenne, l ancienneté est de 798.9578 jours et la popularite de 3.7142 clicks


In [30]:
total_cat, pourcentage_cat, moyenne_anciennete, moyenne_popularite = \
            evaluate_system(model_name='CONTENT',
                            model = content_based_model,
                            sparse_user_item=None,
                            interactions_user_df=interactions_user_df,
                            articles_new_features_df=articles_new_features_df,
                            users_size=USER_SIZE,
                            users=users_list)
print(f'Il y a au total {total_cat} catégories identiques ce qui repréente (sur les recommandations) {pourcentage_cat}%')
print(f'En moyenne, l ancienneté est de {moyenne_anciennete} jours et la popularite de {moyenne_popularite} clicks')

Il y a au total 6959 catégories identiques ce qui repréente (sur les recommandations) 76.4%
En moyenne, l ancienneté est de 643.0816 jours et la popularite de 164.9446 clicks


In [31]:
total_cat, pourcentage_cat, moyenne_anciennete, moyenne_popularite = \
            evaluate_system(model_name='COLLABORATIVE',
                            model=colab_model,
                            sparse_user_item=sparse_user_item,
                            interactions_user_df=interactions_user_df,
                            articles_new_features_df=articles_new_features_df,
                            users_size=USER_SIZE,
                            users=users_list)

print(f'Il y a au total {total_cat} catégories identiques ce qui repréente (sur les recommandations) {pourcentage_cat}%')
print(f'En moyenne, l ancienneté est de {moyenne_anciennete} jours et la popularite de {moyenne_popularite} clicks')

Il y a au total 3681 catégories identiques ce qui repréente (sur les recommandations) 40.4%
En moyenne, l ancienneté est de 510.8694 jours et la popularite de 11460.5756 clicks
